In [1]:
#This program updates the attendance time logs. It detects what the actual shift is rendered

import time, datetime
import openpyxl
wb = openpyxl.load_workbook(r'C:\Users\v.t.flores\Documents\December 31 - January 2 timelogs.xlsx')
sheet = wb['Sheet1']

In [129]:
#Create a new column with the concatenated date and time as values.
for i in range (2, sheet.max_row+1):
    date_str = str(sheet['C' + str(i)].value).split(" ")[0] + " "+ str(sheet['D' + str(i)].value)
    date_date = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    sheet.cell(row=i, column=8).value = date_date

In [173]:
# convTimeIn fcn------------------------------------------------------
def convTimeIn(timeIn):
    hr = timeIn.strftime('%H:%M:%S').split(":")[0]
    mint = timeIn.strftime('%H:%M:%S').split(":")[1]
    sec = timeIn.strftime('%H:%M:%S').split(":")[2]
    newTime = datetime.timedelta(hours=int(hr), minutes=int(mint), seconds=int(sec))
    
    return newTime
#-------------------------------------------------------------


# nearestTimeIn Fcn-------------------------------------------
def nearestTimeIn(timeIn): #This fcn compares the time-in to the nearest equivalent shift
    
    if timeIn < datetime.time(1,30,0):
        return '_1st'        
    else:
        _2nd = datetime.timedelta(hours=7)
        _3ET = datetime.timedelta(hours=11)
        _3rd = datetime.timedelta(hours=15)
        _1ET = datetime.timedelta(hours=19)
        _1st = datetime.timedelta(hours=23)
        _2ET = datetime.timedelta(hours=3)

        tFrom_2nd = abs(convTimeIn(timeIn)-_2nd)
        tFrom_3ET = abs(convTimeIn(timeIn)-_3ET)
        tFrom_3rd = abs(convTimeIn(timeIn)-_3rd)
        tFrom_1ET = abs(convTimeIn(timeIn)-_1ET)
        tFrom_1st = abs(convTimeIn(timeIn)-_1st)
        tFrom_2ET = abs(convTimeIn(timeIn)-_2ET)

        timeInList = [(tFrom_2nd), (tFrom_3ET), (tFrom_3rd), (tFrom_1ET), (tFrom_1st), (tFrom_2ET)]

        probShiftlist = {str(tFrom_2nd): '_2nd', str(tFrom_3ET): '_3ET', str(tFrom_3rd):'_3rd', str(tFrom_1ET): '_1ET', str(tFrom_1st):'_1st', str(tFrom_2ET):'_2ET'}

        return probShiftlist[str(sorted(timeInList)[0])]
#----------------------------------------------------------------    



#nearestTimeOut Fcn---------------------------------------------
def nearestTimeOut(timeOut): #This fcn compares the time-out to the nearest equivalent shift
    
    if timeOut < datetime.time(1,30,0):
        return '_3OT'
    
    else:
        #list of timeouts
        _2nd = datetime.timedelta(hours=15)
        _3rd = datetime.timedelta(hours=23)
        _1st = datetime.timedelta(hours=7)
        _2OT = datetime.timedelta(hours=19)
        _3OT = datetime.timedelta(hours=3)
        _1OT = datetime.timedelta(hours=11)

        tFrom_2nd_out = abs(convTimeIn(timeOut)-_2nd)
        tFrom_3rd_out = abs(convTimeIn(timeOut)-_3rd)
        tFrom_1st_out = abs(convTimeIn(timeOut)-_1st)
        tFrom_2OT_out = abs(convTimeIn(timeOut)-_2OT)
        tFrom_3OT_out = abs(convTimeIn(timeOut)-_3OT)
        tFrom_1OT_out = abs(convTimeIn(timeOut)-_1OT)



        timeOutList = [tFrom_2nd_out, tFrom_3rd_out, tFrom_1st_out, 
                       tFrom_2OT_out, tFrom_3OT_out, tFrom_1OT_out]

        probShiftlist = {str(tFrom_2nd_out): '_2nd', str(tFrom_3rd_out): '_3rd', 
                         str(tFrom_1st_out): '_1st', str(tFrom_2OT_out): '_2OT', 
                         str(tFrom_3OT_out): '_3OT', str(tFrom_1OT_out): '_1OT'}


        return probShiftlist[str(sorted(timeOutList)[0])]
    
#-----------------------------------------------------


#Computing and storing the actual schedule to column L---------------------------------------------------

for in_ in range(2, sheet.max_row + 1):
    for out_ in range(2, sheet.max_row + 1):
        if sheet['E' + str(in_)].value == 'IN'and sheet['E' + str(out_)].value == 'OUT':
            if sheet['A' + str(in_)].value == sheet['A' + str(out_)].value:
                if out_>in_:
                    #computing for hours worked
                    dif = sheet['H' + str(out_)].value - sheet['H' + str(in_)].value
                    sheet['K' + str(out_)].value = dif
                
                    schd_in = nearestTimeIn(sheet['D' + str(in_)].value) #Estimated schedule based on timeIn
                    schd_out = nearestTimeOut(sheet['D' + str(out_)].value) #Estimated schedule based on timeOut

                    t_in = sheet['D' + str(in_)].value #TimeIn actual
                    t_out = sheet['D' + str(out_)].value #TimeOut actual
                    
                    cell_schd = sheet['L' + str(out_)].value
                                    
                    #2nd shift
                    if schd_in == '_2nd' and schd_out == '_2nd':
                        if t_in > datetime.time(hour=7):
                            print('2L')
                            sheet['L' + str(out_)].value = '2L'
                        if t_out < datetime.time(hour=15):
                            print('2UT')
                            sheet['L' + str(out_)].value = '2UT'
                        else:
                            print('2')
                            sheet['L' + str(out_)].value ='2'
                            
                    #2nd shift Undertime
                    if schd_in == '_2nd' and schd_out == '_1OT':
                        if t_in < datetime.time(hour=7) and t_out < datetime.time(hour=15):
                            print('2UT')
                            sheet['L' + str(out_)].value = '2UT'

                    
                    #2nd shift OT
                    if schd_in == '_2nd' and schd_out == '_2OT':
                        if t_in > datetime.time(hour=7):
                            print('2L')
                            sheet['L' + str(out_)].value = '2L'
                        if t_out > datetime.time(hour=19):
                            print('2OT')
                            sheet['L' + str(out_)].value = '2OT'
                        if t_out > datetime.time(hour=15) and t_out < datetime.time(hour=19):
                            print('2')
                            sheet['L' + str(out_)].value = '2'
                    
                    #2nd shift ET
                    if schd_in == '_2ET' and schd_out == '_2nd':
                        if t_in < datetime.time(hour=3) and t_out > datetime.time(hour=15):
                            print('2ET')
                            sheet['L' + str(out_)].value = '2ET'
                        if t_out < datetime.time(hour=15):
                            print('2UT')
                            sheet['L' + str(out_)].value ='2UT'
                            
                    if schd_in == '_2nd' and schd_out == '_2nd':
                        if t_in < datetime.time(hour=7) and t_out < datetime.time(hour=15):
                            print('2UT')
                            sheet['L' + str(out_)].value = '2UT'
                  
                    
                    #2 shift Late
                    if schd_in == '_3ET' and schd_out == '_2nd':                                           
                            print('2L')
                            sheet['L' + str(out_)].value = '2L'                   
                                                                         
                                        
                    #3rd shift
                    if schd_in == '_3rd' and schd_out == '_3rd':
                        if t_in > datetime.time(hour=15):
                            print('3L')
                            sheet['L' + str(out_)].value = '3L'
                        if t_out < datetime.time(hour=23):
                            print('3UT')
                            sheet['L' + str(out_)].value = '3UT'
                        if t_in < datetime.time(hour=15) and t_out > datetime.time(hour=23):
                            print('3')  
                            sheet['L' + str(out_)].value = '3'
                        if t_in < datetime.time(hour=15) and schd_out == '_3rd':
                            print('3')  
                            sheet['L' + str(out_)].value = '3'
                    
                    #3rd shift ET
                    if schd_in == '_3ET' and schd_out == '_3rd':
                        
                        if t_in > datetime.time(hour=11):
                            print('3')
                            sheet['L' + str(out_)].value = '3'
                        if t_out < datetime.time(hour=23):
                            print('3UT')
                            sheet['L' + str(out_)].value = '3UT'
                        if t_in < datetime.time(hour=11) and t_out > datetime.time(hour=23):
                            print('3ET') 
                            sheet['L' + str(out_)].value = '3ET'
                    
                    #3rd shift OT
                    if schd_in == '_3rd' and schd_out == '_3OT':
                        
                        if t_in > datetime.time(hour=15):
                            print('3L')
                            sheet['L' + str(out_)].value = '3L'                   
                        if t_in < datetime.time(hour=15) and t_out > datetime.time(hour=3):
                            print('3OT') 
                            sheet['L' + str(out_)].value = '3OT'
                        if t_in < datetime.time(hour=15) and t_out < datetime.time(hour=3):
                            print('3') 
                            sheet['L' + str(out_)].value = '3'
                            
                    #3rd shift Late
                    if schd_in == '_1ET' and schd_out == '_3rd':                        
                        if t_in > datetime.time(hour=15):
                            print('3L')
                            sheet['L' + str(out_)].value = '3L'     
                    
                    #3rd shift Undertime
                    if schd_in == '_3rd' and schd_out == '_2OT':                        
                        if t_out < datetime.time(hour=23):
                            print('3UT')
                            sheet['L' + str(out_)].value = '3UT'    
                                                 
                    #3rd shift OT,ET
                    if schd_in == '_3ET' and schd_out == '_3OT':
                        
                        if t_in < datetime.time(hour=11) and t_out < datetime.time(hour=3):
                            print('3ET')
                            sheet['L' + str(out_)].value = '3ET'  
                        if t_in < datetime.time(hour=11) and t_out > datetime.time(hour=3):
                            print('3ET+OT')
                            sheet['L' + str(out_)].value = '3ET+OT'
             
                            
                    #1st shift
                    if schd_in == '_1st' and schd_out == '_1st':
                        
                        if t_in > datetime.time(hour=23):
                            print('1L')
                            sheet['L' + str(out_)].value = '1L'
                        if t_out < datetime.time(hour=7):
                            print('1UT')
                            sheet['L' + str(out_)].value = '1UT'
                        if t_in < datetime.time(hour=23) and t_out > datetime.time(hour=7):
                            print('1') 
                            sheet['L' + str(out_)].value = '1'
                    
                    #1st shift ET
                    if schd_in == '_1ET' and schd_out == '_1st':
                        
                        if t_in > datetime.time(hour=19) and t_out > datetime.time(hour=7):
                            print('1')
                            sheet['L' + str(out_)].value = '1'
                        if t_out < datetime.time(hour=7):
                            print('1UT')
                            sheet['L' + str(out_)].value = '1UT'
                        if t_in < datetime.time(hour=19) and t_out > datetime.time(hour=7):
                            print('1ET')
                            sheet['L' + str(out_)].value = '1ET'
                            
                    #1st shift OT
                    if schd_in == '_1st' and schd_out == '_1OT':
                        
                        if t_in > datetime.time(hour=23):
                            print('1L')
                            sheet['L' + str(out_)].value = '1L'
                        if t_out < datetime.time(hour=7):
                            print('1UT')
                            sheet['L' + str(out_)].value = '1UT'
                        if t_in < datetime.time(hour=23) and t_out > datetime.time(hour=11):
                            print('1OT') 
                            sheet['L' + str(out_)].value = '1OT'
                        if t_in < datetime.time(hour=23) and t_out < datetime.time(hour=11):
                            print('1') 
                            sheet['L' + str(out_)].value = '1'

                    #1st shift UT
                    if schd_in == '_1st' and schd_out == '_3OT':                        
                        if t_out < datetime.time(hour=7):
                            print('1UT')
                            sheet['L' + str(out_)].value = '1UT'
                    if schd_in == '_1st' and schd_out == '_2nd':                        
                        if t_out < datetime.time(hour=7):
                            print('1UT')
                            sheet['L' + str(out_)].value = '1UT'
                        
#----------------------------------------------------------------------------------
# Save to a new Excel file
wb.save(r'C:\Users\v.t.flores\Documents\DTR_algo2i.xlsx')

#------End of program --------------------------------


1
1
1
2OT
2OT
2OT
2OT
2OT
2OT
1
3ET
3ET
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2OT
2
2
2
2
2ET
3
3
1ET
1ET
1ET
3
3
3
1
1
1
2OT
2OT
2OT
2OT
2OT
2OT
1
1
1ET
1ET
1ET
1ET
3OT
3OT
3OT
3
3
3
3
3
1
1
1
3
3
3
3
3L
2
2OT
2
2L
2OT
2L
2
2
2
2
2
2
2
2
3
3
3
3
3
3
2
2
2OT
2
2OT
2OT
3
3
2
2OT
2
2OT
2
2
2
2
2
2
2
2
2L
2OT
2L
2OT
2L
2OT
2L
2OT
2L
2OT
2OT
3ET
3ET
3
1ET
1ET
1
3
3
3
3
2L
2L
2L
2
2
2
3
3
3
3
1
1
3
3
3
1ET
1ET
2UT
1
1
1
3
3
3
3
2
2
2
2L
2
2L
2
2
2OT
1
1
1
2
2
2OT
2
2OT
2
2
3
3
3ET
3
3
2OT
2OT
2
2OT
2
2
3
3
3
3
3
2OT
2OT
2
2OT
2
2
3
3
2OT
2
2
3
3
3ET
1OT
1OT
1OT
3
3
3
3
3L
1OT
1
1
1
1
1
3L
3
3
1
2
2OT
2
2OT
2
2
3UT
1
1
1
1
3
3
3
3
3
3
1
1
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
2OT
1ET
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1ET
3ET
3ET
3
2
2
2
2
2
2
1
1
1
1ET
1ET
1ET
1OT
1
1
1
1
1ET
2
2
2
2
2
2
2OT
2OT
2
2OT
2
2L
2
2
2OT
2OT
3ET
3
3
3
3
3
3
2
2
2OT
2
2OT
2OT
3
3
3
3
3
3
2OT
2OT
2OT
1
1UT
1UT
1OT
1OT
1OT
3ET
3ET
3ET
1OT
1OT
1OT
3ET
3ET
3
3
1ET
1ET
1
3
3
3ET
3ET
3
3
1
1
1
1
1OT
1OT
3ET
3ET
3
2
1ET
1ET
1ET
3ET
3ET


In [172]:
wb.save(r'C:\Users\v.t.flores\Documents\DTR_algo2i.xlsx')